In [70]:
from datetime import datetime
import os
import tweepy as tw
import pandas as pd
from geotext import GeoText
import nltk
import spacy
import locationtagger
import matplotlib.pyplot as plt
from branca.element import Figure
import folium
import datetime
import pytz

In [71]:
consumer_key = "L45cVwt1PcimfFnycMhQEHRsg"
consumer_secret = "a2f3dr6xHHukbCzlvzK5j0mDYg3di3aN8EfixXOhzV9dbhMk1F"
access_token = "1533339797187883008-A5uEjCSsKsH4OBeTaO8g0vHaqoI8h6"
access_token_secret = "UKvwBMeTRN6p4f2ogcrQAQT52Fi6eWUhe2pCnEkaJ8UGq"

In [72]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [73]:
utc=pytz.UTC

In [74]:
# Define the search term and the date_since date as variables
search_words = "#wildfire OR #wildfires OR #forestfire OR #bushfire"
startDate = datetime.datetime(2022, 8, 25, 0, 0, 0)
endDate =   datetime.datetime(2022, 8, 27, 0, 0, 0)


In [75]:
startDate = utc.localize(startDate) 
endDate = utc.localize(endDate) 

In [76]:
print(startDate)
print(endDate)

2022-08-25 00:00:00+00:00
2022-08-27 00:00:00+00:00


In [77]:
new_search = search_words + " -filter:retweets"
new_search

'#wildfire OR #wildfires OR #forestfire OR #bushfire -filter:retweets'

In [78]:
tweets = []
tmpTweets = api.search_tweets(new_search)

for tweet in tmpTweets:
    if tweet.created_at < endDate and tweet.created_at > startDate:
        tweets.append(tweet)

while (tmpTweets[-1].created_at > startDate):
    tmpTweets = api.search_tweets(new_search, max_id = tmpTweets[-1].id)
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)


In [79]:
all_tweets = [[tweet.text] for tweet in tweets]

In [80]:
tweet_text = pd.DataFrame(data=all_tweets, 
                    columns=["tweet"])
                    
tweet_text

,tweet
0,The familiar 'watermelon' fire danger ratings ...
1,New Article out: \nRat-Holing Resources. An Et...
2,Can #Wildfires trigger smoke #Allergies? If so...
3,Drones are being used to extinguish🧯 forest fi...
4,How are the elderly affected by California #wi...
...,...
440,NT Bushfire Watch &amp; Act for Vanderlin Driv...
441,#California: Will PG&amp;E get through #wildfi...
442,In response to the increasing number of #wildf...
443,Brown Gold Rust Long Ribbed Knit Hand Knit War...


In [81]:
df = tweet_text[tweet_text["tweet"].str.contains("RT") == False]
df

,tweet
0,The familiar 'watermelon' fire danger ratings ...
1,New Article out: \nRat-Holing Resources. An Et...
2,Can #Wildfires trigger smoke #Allergies? If so...
3,Drones are being used to extinguish🧯 forest fi...
4,How are the elderly affected by California #wi...
...,...
440,NT Bushfire Watch &amp; Act for Vanderlin Driv...
441,#California: Will PG&amp;E get through #wildfi...
442,In response to the increasing number of #wildf...
443,Brown Gold Rust Long Ribbed Knit Hand Knit War...


In [82]:
df['tweet'] = df['tweet'].str.title()
df

/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_28484/3597067682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'] = df['tweet'].str.title()


,tweet
0,The Familiar 'Watermelon' Fire Danger Ratings ...
1,New Article Out: \nRat-Holing Resources. An Et...
2,Can #Wildfires Trigger Smoke #Allergies? If So...
3,Drones Are Being Used To Extinguish🧯 Forest Fi...
4,How Are The Elderly Affected By California #Wi...
...,...
440,Nt Bushfire Watch &Amp; Act For Vanderlin Driv...
441,#California: Will Pg&Amp;E Get Through #Wildfi...
442,In Response To The Increasing Number Of #Wildf...
443,Brown Gold Rust Long Ribbed Knit Hand Knit War...


In [83]:

df_countries = pd.DataFrame()


df_regions = pd.DataFrame()


df_cities = pd.DataFrame()


for index, row in df.iterrows():
    countries = row['tweet']
    place_entity = locationtagger.find_locations(text = countries)
    if place_entity.countries:
     df_countries = df_countries.append({'country': place_entity.countries}, ignore_index=True)
    if place_entity.regions:
     df_regions = df_regions.append({'region': place_entity.regions}, ignore_index=True)
    if place_entity.cities:
        for city_name in place_entity.cities:
            df_cities = df_cities.append({'city': city_name}, ignore_index=True)

/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_28484/3171994184.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities = df_cities.append({'city': city_name}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_28484/3171994184.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_regions = df_regions.append({'region': place_entity.regions}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_28484/3171994184.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities = df_cities.append({'city': city_name}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_28484/3171994184.py:19: FutureWarning: The frame.append method is

In [84]:
df_cities

df_cities.drop_duplicates(subset=['city'], keep=False)

,city
1,Los Angeles
3,Mexico
4,House
6,Canada
7,Valley
8,Hard
9,Grand
12,Singer
16,Advance
19,Colorado Springs


In [102]:
df_cities['frequency'] = df_cities.groupby('city')['city'].transform('size')
df_cities_ft = df_cities.drop_duplicates()
df_cities_ft

,city,frequency
0,Forest,4
1,Los Angeles,1
2,California,19
3,Mexico,1
4,House,1
5,Early,2
6,Canada,1
7,Valley,1
8,Hard,1
9,Grand,1


In [86]:
from geopy.geocoders import Nominatim

loc = Nominatim(user_agent="cal")
print(loc)

In [119]:
df_cities_map = pd.DataFrame()

for ind in df_cities_ft.index:
    getloc = loc.geocode(df_cities_ft['city'][ind])
    df_cities_map = df_cities_map.append({'city': df_cities_ft['city'][ind], 'frequency': df_cities_ft['frequency'][ind] ,'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)

df_cities_map=df_cities_map.dropna()
df_cities_map

/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_28484/2027462833.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_cities_ft['city'][ind], 'frequency': df_cities_ft['frequency'][ind] ,'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_28484/2027462833.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_cities_ft['city'][ind], 'frequency': df_cities_ft['frequency'][ind] ,'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_28484/2027462833.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in 

,city,frequency,latitude,longitude
0,Forest,4,41.532976,-79.264091
1,Los Angeles,1,34.053691,-118.242766
2,California,19,36.701463,-118.755997
3,Mexico,1,23.658512,-102.007710
4,House,1,34.656401,-103.900502
5,Early,2,31.345434,-84.928206
6,Canada,1,61.066692,-107.991707
7,Valley,1,41.559572,-98.980548
8,Hard,1,47.489391,9.691654
9,Grand,1,48.386104,5.486625


In [122]:

# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")


for ind in df_cities_map.index:
    location = geolocator.reverse(df_cities_map['latitude'][ind]+","+df_cities_map['longitude'][ind])
    address = location.raw['address']
    df_cities_map = df_cities_map.append({'country': address.get('country', '')}, ignore_index=True)

df_cities_map

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U1')) -> None

In [112]:
location_data = df_cities_map[['city', 'frequency' ]]
location_data

,city,frequency
0,Forest,4
1,Los Angeles,1
2,California,19
3,Mexico,1
4,House,1
5,Early,2
6,Canada,1
7,Valley,1
8,Hard,1
9,Grand,1


In [113]:
location_geo = df_cities_map[['city', 'latitude', 'longitude' ]]
location_geo

,city,latitude,longitude
0,Forest,41.532976,-79.264091
1,Los Angeles,34.053691,-118.242766
2,California,36.701463,-118.755997
3,Mexico,23.658512,-102.007710
4,House,34.656401,-103.900502
5,Early,31.345434,-84.928206
6,Canada,61.066692,-107.991707
7,Valley,41.559572,-98.980548
8,Hard,47.489391,9.691654
9,Grand,48.386104,5.486625


In [ ]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

m